<a href="https://colab.research.google.com/github/dudaholandah/NLP/blob/main/Projeto/Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Google Drive

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!cp -r '/content/drive/MyDrive/Datasets/cuisine-classification-ingredients' 'cuisine-classification-ingredients'

In [3]:
!cp -r '/content/drive/MyDrive/Datasets/indian-food' 'indian-food'

# Imports

In [4]:
!pip install transformers
!pip install keras
!pip install tensorflow
!pip install keras-tuner --upgrade

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.7 MB 5.1 MB/s 
     |████████████████████████████████| 6.6 MB 41.9 MB/s 
     |████████████████████████████████| 120 kB 62.0 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 135 kB 5.1 MB/s 
     |████████████████████████████████| 1.6 MB 36.2 MB/s 


In [5]:
#TFIDF
from sklearn.feature_extraction.text import TfidfVectorizer
#WORD2VEC
from gensim.models import Word2Vec
import nltk
nltk.download('punkt')
#BERT
import tensorflow as tf
from transformers import BertTokenizer, TFBertModel
#Neural Network
import keras_tuner as kt
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense,Conv1D,Embedding,GlobalMaxPooling1D,MaxPooling1D,Activation,Flatten,SimpleRNN,Dropout
from sklearn.feature_extraction.text import TfidfVectorizer
from tensorflow.keras.optimizers import SGD
from keras_tuner import HyperModel
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report
#Data
import json
import os
import pandas as pd
import re
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.utils.np_utils import to_categorical

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


# Data

Cuisine Classification Data

In [6]:
path = 'cuisine-classification-ingredients/train.json'

with open(path, 'r') as f:
  json = json.load(f)

In [7]:
df_cuisine = pd.DataFrame(json, columns=['id', 'cuisine', 'ingredients'])

In [8]:
df_cuisine = df_cuisine.drop(['id'], axis=1)

In [9]:
df_cuisine = df_cuisine[:1000]

In [10]:
df_cuisine.head(5)

,cuisine,ingredients
0,greek,"[romaine lettuce, black olives, grape tomatoes..."
1,southern_us,"[plain flour, ground pepper, salt, tomatoes, g..."
2,filipino,"[eggs, pepper, salt, mayonaise, cooking oil, g..."
3,indian,"[water, vegetable oil, wheat, salt]"
4,indian,"[black pepper, shallots, cornflour, cayenne pe..."


Indian Food Data

In [11]:
path = 'indian-food/indian_food.csv'
df_indian = pd.read_csv(path)

In [12]:
df_indian.head(5)

,name,ingredients,diet,prep_time,cook_time,flavor_profile,course,state,region
0,Balu shahi,"Maida flour, yogurt, oil, sugar",vegetarian,45,25,sweet,dessert,West Bengal,East
1,Boondi,"Gram flour, ghee, sugar",vegetarian,80,30,sweet,dessert,Rajasthan,West
2,Gajar ka halwa,"Carrots, milk, sugar, ghee, cashews, raisins",vegetarian,15,60,sweet,dessert,Punjab,North
3,Ghevar,"Flour, ghee, kewra, milk, clarified butter, su...",vegetarian,15,30,sweet,dessert,Rajasthan,West
4,Gulab jamun,"Milk powder, plain flour, baking powder, ghee,...",vegetarian,15,40,sweet,dessert,West Bengal,East


## Pre-Processing

In [13]:
def pre_processing(text):
  new = ""
  for word in text.split(','):
    word = re.sub(r'[^\w\s]', '', word.lower())
    word = re.sub(r'[0-9]+', '', word)
    word = re.sub(r'\boz\b', '', word)
    word = re.sub(r'\ba taste of\b', '', word)
    new = new + word.strip() + " "

  return new[:-1]

Cuisine Classification Data

In [14]:
df_cuisine['ingredients'] = [','.join(x).strip() for x in df_cuisine['ingredients']]

In [15]:
df_cuisine['ingredients'] = [pre_processing(x) for x in df_cuisine['ingredients']]

In [16]:
X = df_cuisine['ingredients']
y = df_cuisine['cuisine']

# dados de teste = 20% do treinamento
X_train_cuisine, X_test_cuisine, y_train_cuisine, y_test_cuisine = train_test_split(X, y, test_size=0.2)

# dados de validacao = 50% do teste
X_test_cuisine, X_valid_cuisine, y_test_cuisine, y_valid_cuisine = train_test_split(X, y, test_size=0.5)

In [17]:
y_train_cuisine = pd.Categorical(y_train_cuisine)
y_train_int_cuisine = y_train_cuisine.codes

y_valid_cuisine = pd.Categorical(y_valid_cuisine)
y_valid_int_cuisine = y_valid_cuisine.codes

y_test_cuisine = pd.Categorical(y_test_cuisine)
y_test_int_cuisine = y_test_cuisine.codes

y_train_cuisine = to_categorical(y_train_int_cuisine)
y_valid_cuisine = to_categorical(y_valid_int_cuisine)
y_test_cuisine = to_categorical(y_test_int_cuisine)

Indian Food Data

In [18]:
df_indian['ingredients'] = [pre_processing(x) for x in df_indian['ingredients']]

In [19]:
X = df_indian['ingredients']
y = df_indian['diet']

# dados de teste = 20% do treinamento
X_train_indian, X_test_indian, y_train_indian, y_test_indian = train_test_split(X, y, test_size=0.2)

# dados de validacao = 50% do teste
X_test_indian, X_valid_indian, y_test_indian, y_valid_indian = train_test_split(X, y, test_size=0.5)

In [20]:
y_train_indian = pd.Categorical(y_train_indian)
y_train_int_indian = y_train_indian.codes

y_valid_indian = pd.Categorical(y_valid_indian)
y_valid_int_indian = y_valid_indian.codes

y_test_indian = pd.Categorical(y_test_indian)
y_test_int_indian = y_test_indian.codes

y_train_indian = to_categorical(y_train_int_indian)
y_valid_indian = to_categorical(y_valid_int_indian)
y_test_indian = to_categorical(y_test_int_indian)

# Different Embeddings

In [21]:
vocabulary_cuisine = {}
i = 0

for ingredients in df_cuisine['ingredients']:
  for word in ingredients.split():
    if word not in vocabulary_cuisine:
      vocabulary_cuisine[word] = i
      i += 1

In [22]:
vocabulary_indian = {}
i = 0

for ingredients in df_indian['ingredients']:
  for word in ingredients.split():
    if word not in vocabulary_indian:
      vocabulary_indian[word] = i
      i += 1

## TFIDF

Cuisine Classification Data

In [23]:
tfidf_vectorizer = TfidfVectorizer(vocabulary=vocabulary_cuisine)

In [24]:
# Training Data
tfidf_train_cuisine = tfidf_vectorizer.fit_transform(X_train_cuisine)
tfidf_train_cuisine = tfidf_train_cuisine.toarray()
df_tfidf_train_cuisine = pd.DataFrame(tfidf_train_cuisine, columns=tfidf_vectorizer.get_feature_names_out())

In [25]:
df_tfidf_train_cuisine.shape

(800, 1211)

In [26]:
# Validation Data
tfidf_valid_cuisine = tfidf_vectorizer.fit_transform(X_valid_cuisine)
tfidf_valid_cuisine = tfidf_valid_cuisine.toarray()
df_tfidf_valid_cuisine = pd.DataFrame(tfidf_valid_cuisine, columns=tfidf_vectorizer.get_feature_names_out())

In [27]:
df_tfidf_valid_cuisine.shape

(500, 1211)

In [28]:
# Test Data
tfidf_test_cuisine = tfidf_vectorizer.fit_transform(X_test_cuisine)
tfidf_test_cuisine = tfidf_test_cuisine.toarray()
df_tfidf_test_cuisine = pd.DataFrame(tfidf_test_cuisine, columns=tfidf_vectorizer.get_feature_names_out())

In [29]:
df_tfidf_test_cuisine.shape

(500, 1211)

Indian Food Data

In [30]:
tfidf_vectorizer = TfidfVectorizer(vocabulary=vocabulary_indian)

In [31]:
# Training Data
tfidf_train_indian = tfidf_vectorizer.fit_transform(X_train_indian)
tfidf_train_indian = tfidf_train_indian.toarray()
df_tfidf_train_indian = pd.DataFrame(tfidf_train_indian, columns=tfidf_vectorizer.get_feature_names_out())

In [32]:
df_tfidf_train_indian.shape

(204, 337)

In [33]:
# Validation Data
tfidf_valid_indian = tfidf_vectorizer.fit_transform(X_valid_indian)
tfidf_valid_indian = tfidf_valid_indian.toarray()
df_tfidf_valid_indian = pd.DataFrame(tfidf_valid_indian, columns=tfidf_vectorizer.get_feature_names_out())

In [34]:
df_tfidf_valid_indian.shape

(128, 337)

In [35]:
# Test Data
tfidf_test_indian = tfidf_vectorizer.fit_transform(X_test_indian)
tfidf_test_indian = tfidf_test_indian.toarray()
df_tfidf_test_indian = pd.DataFrame(tfidf_test_indian, columns=tfidf_vectorizer.get_feature_names_out())

In [36]:
df_tfidf_test_indian.shape

(127, 337)

## Word2Vec

In [37]:
def get_embedding_w2v(model, doc_tokens):
    embeddings = []
    for tok in doc_tokens:
        if tok in model.wv.vocab:
                embeddings.append(model.wv.word_vec(tok))
        else:
            embeddings.append(np.random.rand(300))
    return np.mean(embeddings, axis=0)

Cuisine Classification Data

In [38]:
# Train Data
ingredients = '. '.join(X_train_cuisine)
sentences = nltk.sent_tokenize(ingredients)
sentences = [nltk.word_tokenize(sentence) for sentence in sentences]
model_word2vec = Word2Vec(sentences, size=300, min_count=1, window=5, sg=1,workers=4)
X_train_word2vec_cuisine = X_train_cuisine.apply(lambda x : get_embedding_w2v(model_word2vec, x.split()))
X_train_word2vec_cuisine = np.array([x.tolist() for x in X_train_word2vec_cuisine])

In [39]:
# Validation Data
ingredients = '. '.join(X_valid_cuisine)
sentences = nltk.sent_tokenize(ingredients)
sentences = [nltk.word_tokenize(sentence) for sentence in sentences]
model_word2vec = Word2Vec(sentences, size=300, min_count=1, window=5, sg=1,workers=4)
X_valid_word2vec_cuisine = X_valid_cuisine.apply(lambda x : get_embedding_w2v(model_word2vec, x.split()))
X_valid_word2vec_cuisine = np.array([x.tolist() for x in X_valid_word2vec_cuisine])

In [40]:
# Test Data
ingredients = '. '.join(X_test_cuisine)
sentences = nltk.sent_tokenize(ingredients)
sentences = [nltk.word_tokenize(sentence) for sentence in sentences]
model_word2vec = Word2Vec(sentences, size=300, min_count=1, window=5, sg=1,workers=4)
X_test_word2vec_cuisine = X_test_cuisine.apply(lambda x : get_embedding_w2v(model_word2vec, x.split()))
X_test_word2vec_cuisine = np.array([x.tolist() for x in X_test_word2vec_cuisine])

Indian Food Data

In [41]:
# Train Data
ingredients = '. '.join(X_train_indian)
sentences = nltk.sent_tokenize(ingredients)
sentences = [nltk.word_tokenize(sentence) for sentence in sentences]
model_word2vec = Word2Vec(sentences, size=300, min_count=1, window=5, sg=1,workers=1)
X_train_word2vec_indian = X_train_indian.apply(lambda x : get_embedding_w2v(model_word2vec, x.split()))
X_train_word2vec_indian = np.array([x.tolist() for x in X_train_word2vec_indian])

In [42]:
# Validation Data
ingredients = '. '.join(X_valid_indian)
sentences = nltk.sent_tokenize(ingredients)
sentences = [nltk.word_tokenize(sentence) for sentence in sentences]
model_word2vec = Word2Vec(sentences, size=300, min_count=1, window=5, sg=1,workers=1)
X_valid_word2vec_indian = X_valid_indian.apply(lambda x : get_embedding_w2v(model_word2vec, x.split()))
X_valid_word2vec_indian = np.array([x.tolist() for x in X_valid_word2vec_indian])

In [43]:
# Test Data
ingredients = '. '.join(X_test_indian)
sentences = nltk.sent_tokenize(ingredients)
sentences = [nltk.word_tokenize(sentence) for sentence in sentences]
model_word2vec = Word2Vec(sentences, size=300, min_count=1, window=5, sg=1,workers=1)
X_test_word2vec_indian = X_test_indian.apply(lambda x : get_embedding_w2v(model_word2vec, x.split()))
X_test_word2vec_indian = np.array([x.tolist() for x in X_test_word2vec_indian])

## BERT

In [44]:
#bert_model_sentence = SentenceTransformer('bert-base-nli-mean-tokens')

In [45]:
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')
model = TFBertModel.from_pretrained("bert-base-multilingual-cased")

Some layers from the model checkpoint at bert-base-multilingual-cased were not used when initializing TFBertModel: ['mlm___cls', 'nsp___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-multilingual-cased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


Cuisine Classification Data

In [46]:
# Train Data
encoded_input = [tokenizer(text, return_tensors='tf') for text in X_train_cuisine]
xs = [model(text) for text in encoded_input]
X_train_bert_cuisine = [text[1][0].numpy() for text in xs]
X_train_bert_cuisine = np.array(X_train_bert_cuisine)

In [47]:
# Validation Data
encoded_input = [tokenizer(text, return_tensors='tf') for text in X_valid_cuisine]
xs = [model(text) for text in encoded_input]
X_valid_bert_cuisine = [text[1][0].numpy() for text in xs]
X_valid_bert_cuisine = np.array(X_valid_bert_cuisine)

In [48]:
# Test Data
encoded_input = [tokenizer(text, return_tensors='tf') for text in X_test_cuisine]
xs = [model(text) for text in encoded_input]
X_test_bert_cuisine = [text[1][0].numpy() for text in xs]
X_test_bert_cuisine = np.array(X_test_bert_cuisine)

Indian Food Data

In [49]:
# Train Data
encoded_input = [tokenizer(text, return_tensors='tf') for text in X_train_indian]
xs = [model(text) for text in encoded_input]
X_train_bert_indian = [text[1][0].numpy() for text in xs]
X_train_bert_indian = np.array(X_train_bert_indian)

In [50]:
# Validation Data
encoded_input = [tokenizer(text, return_tensors='tf') for text in X_valid_indian]
xs = [model(text) for text in encoded_input]
X_valid_bert_indian = [text[1][0].numpy() for text in xs]
X_valid_bert_indian = np.array(X_valid_bert_indian)

In [51]:
# Test Data
encoded_input = [tokenizer(text, return_tensors='tf') for text in X_test_indian]
xs = [model(text) for text in encoded_input]
X_test_bert_indian = [text[1][0].numpy() for text in xs]
X_test_bert_indian = np.array(X_test_bert_indian)

# Different Classificators

In [52]:
vocab_size_cuisine = len(vocabulary_cuisine)
vocab_size_indian = len(vocabulary_indian)

## CNN

In [53]:
class CNNHyperModel(HyperModel):
  def __init__(self, vocab_size, max_length, output):
    self.vocab_size = vocab_size
    self.max_length = max_length
    self.output = output

  def build(self, hp):
    model = Sequential()
    model.add(Embedding(self.vocab_size, output_dim = 32, input_length=self.max_length))
    model.add(Conv1D(filters=hp.Int("conv_1", min_value=32, max_value=96, step=32), kernel_size=11, strides=1))
    model.add(Activation('relu'))
    model.add(GlobalMaxPooling1D())
    model.add(Dense(hp.Choice("dense_units", [64,128,256])))
    model.add(Activation('relu'))
    model.add(Dense(hp.Choice("dense_units2", [16,32, 64])))
    model.add(Activation('relu'))
    model.add(Dense(self.output, activation='softmax'))
    sgd = SGD(learning_rate=hp.Choice("learning_rt", [0.01,0.001,0.001]))
    model.compile(loss="categorical_crossentropy",optimizer=sgd,metrics=['accuracy'])
    return model

### TFIDF

Cuisine Classification Data

In [54]:
model_CNN = CNNHyperModel(vocab_size_cuisine, vocab_size_cuisine, 20)

In [55]:
tuner = kt.Hyperband(
    model_CNN,
    objective='val_accuracy',
    max_epochs=30,
    hyperband_iterations=2,
    overwrite=True)

In [56]:
tuner.search(df_tfidf_train_cuisine,y_train_cuisine,validation_data=(df_tfidf_valid_cuisine,y_valid_cuisine), epochs=10)

Trial 66 Complete [00h 00m 21s]
val_accuracy: 0.20000000298023224

Best val_accuracy So Far: 0.23399999737739563
Total elapsed time: 00h 21m 30s

Search: Running Trial #67

Value             |Best Value So Far |Hyperparameter
96                |64                |conv_1
64                |64                |dense_units
64                |64                |dense_units2
0.01              |0.001             |learning_rt
4                 |2                 |tuner/epochs
0                 |0                 |tuner/initial_epoch
2                 |3                 |tuner/bracket
0                 |0                 |tuner/round

Epoch 1/4
25/25 [==============================] - 7s 256ms/step - loss: 2.9810 - accuracy: 0.1775 - val_loss: 2.9652 - val_accuracy: 0.2340
Epoch 2/4
13/25 [==============>...............] - ETA: 2s - loss: 2.9607 - accuracy: 0.2115

KeyboardInterrupt: ignored

In [ ]:
bestHP = tuner.get_best_hyperparameters()[0]
print("[INFO] optimal number of filters in conv_1 layer: {}".format(bestHP.get("conv_1")))
print("[INFO] optimal number of units in dense layer: {}".format(bestHP.get("dense_units")))
print("[INFO] optimal number of units in dense layer 2: {}".format(bestHP.get("dense_units2")))
print("[INFO] optimal learning rate: {}".format(bestHP.get("learning_rt")))

In [ ]:
model_cuisine = tuner.get_best_models()[0]

In [ ]:
history_fine = model_cuisine.fit(df_tfidf_train_cuisine,y_train_cuisine,validation_data=(df_tfidf_valid_cuisine,y_valid_cuisine), epochs=10)

In [ ]:
plt.plot(history_fine.history['loss'])
plt.plot(history_fine.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['training set','validation set'], loc='upper left')
plt.show()

In [ ]:
plt.plot(history_fine.history['accuracy'])
plt.plot(history_fine.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['training set','validation set'], loc='upper left')
plt.show()

In [ ]:
y_prob = model_cuisine.predict(df_tfidf_test_cuisine)
y_pred = np.argmax(y_prob,axis=1)
print(classification_report(y_test_int_cuisine,y_pred))

Indian Food Data

In [ ]:
model_CNN = CNNHyperModel(vocab_size_indian, vocab_size_indian, 2)

In [ ]:
tuner = kt.Hyperband(
    model_CNN,
    objective='val_accuracy',
    max_epochs=30,
    hyperband_iterations=2, 
    overwrite=True)

In [ ]:
tuner.search(df_tfidf_train_indian,y_train_indian,validation_data=(df_tfidf_valid_indian,y_valid_indian), epochs=10)

In [ ]:
bestHP = tuner.get_best_hyperparameters()[0]
print("[INFO] optimal number of filters in conv_1 layer: {}".format(bestHP.get("conv_1")))
print("[INFO] optimal number of units in dense layer: {}".format(bestHP.get("dense_units")))
print("[INFO] optimal number of units in dense layer 2: {}".format(bestHP.get("dense_units2")))
print("[INFO] optimal learning rate: {}".format(bestHP.get("learning_rt")))

In [ ]:
model_indian = tuner.get_best_models()[0]

In [ ]:
history_fine = model_indian.fit(df_tfidf_train_indian,y_train_indian,validation_data=(df_tfidf_valid_indian,y_valid_indian), epochs=10)

In [ ]:
plt.plot(history_fine.history['loss'])
plt.plot(history_fine.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['training set','validation set'], loc='upper left')
plt.show()

In [ ]:
plt.plot(history_fine.history['accuracy'])
plt.plot(history_fine.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['training set','validation set'], loc='upper left')
plt.show()

In [ ]:
y_prob = model_indian.predict(df_tfidf_test_indian)
y_pred = np.argmax(y_prob,axis=1)
print(classification_report(y_test_int_indian,y_pred))

### Word2vec

Cuisine Classification Data

In [ ]:
X_train_word2vec_cuisine = np.array([np.array(val) for val in X_train_word2vec_cuisine])
X_valid_word2vec_cuisine = np.array([np.array(val) for val in X_valid_word2vec_cuisine])

In [ ]:
model_CNN = CNNHyperModel(vocab_size_cuisine, 300, 20)

In [ ]:
tuner = kt.Hyperband(
    model_CNN,
    objective='val_accuracy',
    max_epochs=30,
    hyperband_iterations=2, 
    overwrite=True)

In [ ]:
tuner.search(X_train_word2vec_cuisine,y_train_cuisine,validation_data=(X_valid_word2vec_cuisine,y_valid_cuisine) , epochs=10)

In [ ]:
model_cuisine = tuner.get_best_models()[0]

In [ ]:
bestHP = tuner.get_best_hyperparameters()[0]
print("[INFO] optimal number of filters in conv_1 layer: {}".format(bestHP.get("conv_1")))
print("[INFO] optimal number of units in dense layer: {}".format(bestHP.get("dense_units")))
print("[INFO] optimal number of units in dense layer 2: {}".format(bestHP.get("dense_units2")))
print("[INFO] optimal learning rate: {}".format(bestHP.get("learning_rt")))

In [ ]:
history_fine = model_cuisine.fit(X_train_word2vec_cuisine,y_train_cuisine,validation_data=(X_valid_word2vec_cuisine,y_valid_cuisine) , epochs=20)

In [ ]:
plt.plot(history_fine.history['loss'])
plt.plot(history_fine.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['training set','validation set'], loc='upper left')
plt.show()

In [ ]:
plt.plot(history_fine.history['accuracy'])
plt.plot(history_fine.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['training set','validation set'], loc='upper left')
plt.show()

In [ ]:
y_prob = model_cuisine.predict(X_test_word2vec_cuisine)
y_pred = np.argmax(y_prob,axis=1)
print(classification_report(y_test_int_cuisine,y_pred))

Indian Food Data

In [ ]:
X_train_word2vec_indian = np.array([np.array(val) for val in X_train_word2vec_indian])
X_valid_word2vec_indian = np.array([np.array(val) for val in X_valid_word2vec_indian])

In [ ]:
model_CNN = CNNHyperModel(vocab_size_indian, 300, 2)

In [ ]:
tuner = kt.Hyperband(
    model_CNN,
    objective='val_accuracy',
    max_epochs=30,
    hyperband_iterations=2, 
    overwrite=True)

In [ ]:
tuner.search(X_train_word2vec_indian,y_train_indian,validation_data=(X_valid_word2vec_indian,y_valid_indian) , epochs=10)

In [ ]:
bestHP = tuner.get_best_hyperparameters()[0]
print("[INFO] optimal number of filters in conv_1 layer: {}".format(bestHP.get("conv_1")))
print("[INFO] optimal number of units in dense layer: {}".format(bestHP.get("dense_units")))
print("[INFO] optimal number of units in dense layer 2: {}".format(bestHP.get("dense_units2")))
print("[INFO] optimal learning rate: {}".format(bestHP.get("learning_rt")))

In [ ]:
model_indian = tuner.get_best_models()[0]

In [ ]:
history_fine = model_indian.fit(X_train_word2vec_indian,y_train_indian,validation_data=(X_valid_word2vec_indian,y_valid_indian) , epochs=20)

In [ ]:
plt.plot(history_fine.history['loss'])
plt.plot(history_fine.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['training set','validation set'], loc='upper left')
plt.show()

In [ ]:
plt.plot(history_fine.history['accuracy'])
plt.plot(history_fine.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['training set','validation set'], loc='upper left')
plt.show()

In [ ]:
y_prob = model_indian.predict(X_test_word2vec_indian)
y_pred = np.argmax(y_prob,axis=1)
print(classification_report(y_test_int_indian,y_pred))

### BERT

Cuisine Classification Data

In [ ]:
model_CNN = CNNHyperModel(vocab_size_cuisine, 768, 20)

In [ ]:
tuner = kt.Hyperband(
    model_CNN,
    objective='val_accuracy',
    max_epochs=30,
    hyperband_iterations=2, 
    overwrite=True)

In [ ]:
tuner.search(X_train_bert_cuisine,y_train_cuisine,validation_data=(X_valid_bert_cuisine,y_valid_cuisine), epochs=10)

In [ ]:
bestHP = tuner.get_best_hyperparameters()[0]
print("[INFO] optimal number of filters in conv_1 layer: {}".format(bestHP.get("conv_1")))
print("[INFO] optimal number of units in dense layer: {}".format(bestHP.get("dense_units")))
print("[INFO] optimal number of units in dense layer 2: {}".format(bestHP.get("dense_units2")))
print("[INFO] optimal learning rate: {}".format(bestHP.get("learning_rt")))

In [ ]:
model_cuisine = tuner.get_best_models()[0]

In [ ]:
history_fine = model_cuisine.fit(X_train_bert_cuisine,y_train_cuisine,validation_data=(X_valid_bert_cuisine,y_valid_cuisine), epochs=10)

In [ ]:
plt.plot(history_fine.history['loss'])
plt.plot(history_fine.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['training set','validation set'], loc='upper left')
plt.show()

In [ ]:
plt.plot(history_fine.history['accuracy'])
plt.plot(history_fine.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['training set','validation set'], loc='upper left')
plt.show()

In [ ]:
y_prob = model_cuisine.predict(X_test_bert_cuisine)
y_pred = np.argmax(y_prob,axis=1)
print(classification_report(y_test_int_cuisine,y_pred))

Indian Food Data

In [ ]:
model_CNN = CNNHyperModel(vocab_size_indian, 768, 2)

In [ ]:
tuner = kt.Hyperband(
    model_CNN,
    objective='val_accuracy',
    max_epochs=30,
    hyperband_iterations=2, 
    overwrite=True)

In [ ]:
tuner.search(X_train_bert_indian,y_train_indian,validation_data=(X_valid_bert_indian,y_valid_indian), epochs=10)

In [ ]:
bestHP = tuner.get_best_hyperparameters()[0]
print("[INFO] optimal number of filters in conv_1 layer: {}".format(bestHP.get("conv_1")))
print("[INFO] optimal number of units in dense layer: {}".format(bestHP.get("dense_units")))
print("[INFO] optimal number of units in dense layer 2: {}".format(bestHP.get("dense_units2")))
print("[INFO] optimal learning rate: {}".format(bestHP.get("learning_rt")))

In [ ]:
model_indian = tuner.get_best_models()[0]

In [ ]:
history_fine = model_indian.fit(X_train_bert_indian,y_train_indian,validation_data=(X_valid_bert_indian,y_valid_indian), epochs=10)

In [ ]:
plt.plot(history_fine.history['loss'])
plt.plot(history_fine.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['training set','validation set'], loc='upper left')
plt.show()

In [ ]:
plt.plot(history_fine.history['accuracy'])
plt.plot(history_fine.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['training set','validation set'], loc='upper left')
plt.show()

In [ ]:
y_prob = model_indian.predict(X_test_bert_indian)
y_pred = np.argmax(y_prob,axis=1)
print(classification_report(y_test_int_indian,y_pred))

## RNN

In [ ]:
from keras_tuner import HyperModel

class RNNHyperModel(HyperModel):
    def __init__(self, vocab_size, max_length, output):
        self.vocab_size = vocab_size
        self.max_length = max_length
        self.output = output

    def build(self, hp):
        model = keras.Sequential()
        model = Sequential()
        model.add(Embedding(self.vocab_size, output_dim = 32, input_length=self.max_length))
        model.add(SimpleRNN(hp.Choice("rnn1", [64,128,256]), return_sequences=True, activation='relu'))
        model.add(Dropout(0.5))
        model.add(SimpleRNN(hp.Choice("rnn2", [16,32,64]), activation='relu'))
        model.add(Dropout(0.5))
        model.add(Dense(hp.Choice("dense_units", [16,32, 64]), activation='relu'))
        model.add(Dropout(0.5))
        model.add(Dense(self.output, activation="softmax"))
        sgd = SGD(learning_rate=hp.Choice("learning_rt", [0.01,0.001,0.001]))
        model.compile(loss="categorical_crossentropy",optimizer=sgd,metrics=['accuracy'])
        return model

### TFIDF

Cuisine Classification Data

In [ ]:
model_RNN = RNNHyperModel(vocab_size_cuisine, vocab_size_cuisine, 20)

In [ ]:
tuner = kt.Hyperband(
    model_RNN,
    objective='val_accuracy',
    max_epochs=30,
    hyperband_iterations=2, 
    overwrite=True)

In [ ]:
tuner.search(df_tfidf_train_cuisine,y_train_cuisine,validation_data=(df_tfidf_valid_cuisine,y_valid_cuisine), epochs=10)

In [ ]:
bestHP = tuner.get_best_hyperparameters()[0]
print("[INFO] optimal number of filters in rnn1 layer: {}".format(bestHP.get("rnn1")))
print("[INFO] optimal number of units in rnn2 layer: {}".format(bestHP.get("rnn2")))
print("[INFO] optimal number of units in dense layer: {}".format(bestHP.get("dense_units")))

In [ ]:
model_cuisine = tuner.get_best_models()[0]

In [ ]:
history_fine = model_cuisine.fit(df_tfidf_train_cuisine,y_train_cuisine,validation_data=(df_tfidf_valid_cuisine,y_valid_cuisine), epochs=10)

In [ ]:
plt.plot(history_fine.history['loss'])
plt.plot(history_fine.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['training set','validation set'], loc='upper left')
plt.show()

In [ ]:
plt.plot(history_fine.history['accuracy'])
plt.plot(history_fine.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['training set','validation set'], loc='upper left')
plt.show()

In [ ]:
y_prob = model_cuisine.predict(df_tfidf_test_cuisine)
y_pred = np.argmax(y_prob,axis=1)
print(classification_report(y_test_int_cuisine,y_pred))

Indian Food Data

In [ ]:
model_RNN = RNNHyperModel(vocab_size_indian, vocab_size_indian, 20)

In [ ]:
tuner = kt.Hyperband(
    model_RNN,
    objective='val_accuracy',
    max_epochs=30,
    hyperband_iterations=2, 
    overwrite=True)

In [ ]:
tuner.search(df_tfidf_train_indian,y_train_indian,validation_data=(df_tfidf_valid_indian,y_valid_indian), epochs=10)

In [ ]:
bestHP = tuner.get_best_hyperparameters()[0]
print("[INFO] optimal number of filters in rnn1 layer: {}".format(bestHP.get("rnn1")))
print("[INFO] optimal number of units in rnn2 layer: {}".format(bestHP.get("rnn2")))
print("[INFO] optimal number of units in dense layer: {}".format(bestHP.get("dense_units")))

In [ ]:
model_indian = tuner.get_best_models()[0]

In [ ]:
history_fine = model_indian.fit(df_tfidf_train_indian,y_train_indian,validation_data=(df_tfidf_valid_indian,y_valid_indian), epochs=10)

In [ ]:
plt.plot(history_fine.history['loss'])
plt.plot(history_fine.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['training set','validation set'], loc='upper left')
plt.show()

In [ ]:
plt.plot(history_fine.history['accuracy'])
plt.plot(history_fine.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['training set','validation set'], loc='upper left')
plt.show()

In [ ]:
y_prob = model_indian.predict(df_tfidf_test_indian)
y_pred = np.argmax(y_prob,axis=1)
print(classification_report(y_test_int_indian,y_pred))

### Word2vec

Cuisine Classification Data

In [ ]:
X_train_word2vec_cuisine = np.array([np.array(val) for val in X_train_word2vec_cuisine])
X_valid_word2vec_cuisine = np.array([np.array(val) for val in X_valid_word2vec_cuisine])

In [ ]:
model_RNN = RNNHyperModel(vocab_size_cuisine, 300, 20)

In [ ]:
tuner = kt.Hyperband(
    model_RNN,
    objective='val_accuracy',
    max_epochs=30,
    hyperband_iterations=2, 
    overwrite=True)

In [ ]:
tuner.search(X_train_word2vec_cuisine,y_train_cuisine,validation_data=(X_valid_word2vec_cuisine,y_valid_cuisine) , epochs=10)

In [ ]:
bestHP = tuner.get_best_hyperparameters()[0]
print("[INFO] optimal number of filters in rnn1 layer: {}".format(bestHP.get("rnn1")))
print("[INFO] optimal number of units in rnn2 layer: {}".format(bestHP.get("rnn2")))
print("[INFO] optimal number of units in dense layer: {}".format(bestHP.get("dense_units")))
print("[INFO] optimal learning rate: {}".format(bestHP.get("learning_rt")))

In [ ]:
model_cuisine = tuner.get_best_models()[0]

In [ ]:
history_fine = model_cuisine.fit(X_train_word2vec_cuisine,y_train_cuisine,validation_data=(X_valid_word2vec_cuisine,y_valid_cuisine) , epochs=20)

In [ ]:
plt.plot(history_fine.history['loss'])
plt.plot(history_fine.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['training set','validation set'], loc='upper left')
plt.show()

In [ ]:
plt.plot(history_fine.history['accuracy'])
plt.plot(history_fine.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['training set','validation set'], loc='upper left')
plt.show()

In [ ]:
y_prob = model_cuisine.predict(X_test_word2vec_cuisine)
y_pred = np.argmax(y_prob,axis=1)
print(classification_report(y_test_int_cuisine,y_pred))

Indian Food Data

In [ ]:
X_train_word2vec_indian = np.array([np.array(val) for val in X_train_word2vec_indian])
X_valid_word2vec_indian = np.array([np.array(val) for val in X_valid_word2vec_indian])

In [ ]:
model_RNN = RNNHyperModel(vocab_size_indian, 300, 2)

In [ ]:
tuner = kt.Hyperband(
    model_RNN,
    objective='val_accuracy',
    max_epochs=30,
    hyperband_iterations=2, 
    overwrite=True)

In [ ]:
tuner.search(X_train_word2vec_indian,y_train_indian,validation_data=(X_valid_word2vec_indian,y_valid_indian), epochs=10)

In [ ]:
bestHP = tuner.get_best_hyperparameters()[0]
print("[INFO] optimal number of filters in rnn1 layer: {}".format(bestHP.get("rnn1")))
print("[INFO] optimal number of units in rnn2 layer: {}".format(bestHP.get("rnn2")))
print("[INFO] optimal number of units in dense layer: {}".format(bestHP.get("dense_units")))
print("[INFO] optimal learning rate: {}".format(bestHP.get("learning_rt")))

In [ ]:
model_indian = tuner.get_best_models()[0]

In [ ]:
history_fine = model_indian.fit(X_train_word2vec_indian,y_train_indian,validation_data=(X_valid_word2vec_indian,y_valid_indian) , epochs=20)

In [ ]:
plt.plot(history_fine.history['loss'])
plt.plot(history_fine.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['training set','validation set'], loc='upper left')
plt.show()

In [ ]:
plt.plot(history_fine.history['accuracy'])
plt.plot(history_fine.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['training set','validation set'], loc='upper left')
plt.show()

In [ ]:
y_prob = model_indian.predict(X_test_word2vec_indian)
y_pred = np.argmax(y_prob,axis=1)
print(classification_report(y_test_int_indian,y_pred))

### BERT

Cuisine Classification Data

In [ ]:
model_RNN = RNNHyperModel(vocab_size_cuisine, 768, 20)

In [ ]:
tuner = kt.Hyperband(
    model_RNN,
    objective='val_accuracy',
    max_epochs=30,
    hyperband_iterations=2, 
    overwrite=True)

In [ ]:
tuner.search(X_train_bert_cuisine,y_train_cuisine,validation_data=(X_valid_bert_cuisine,y_valid_cuisine), epochs=10)

In [ ]:
bestHP = tuner.get_best_hyperparameters()[0]
print("[INFO] optimal number of filters in rnn1 layer: {}".format(bestHP.get("rnn1")))
print("[INFO] optimal number of units in rnn2 layer: {}".format(bestHP.get("rnn2")))
print("[INFO] optimal number of units in dense layer: {}".format(bestHP.get("dense_units")))
print("[INFO] optimal learning rate: {}".format(bestHP.get("learning_rt")))

In [ ]:
model_cuisine = tuner.get_best_models()[0]

In [ ]:
history_fine = model_cuisine.fit(X_train_bert_cuisine,y_train_cuisine,validation_data=(X_valid_bert_cuisine,y_valid_cuisine), epochs=10)

In [ ]:
plt.plot(history_fine.history['loss'])
plt.plot(history_fine.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['training set','validation set'], loc='upper left')
plt.show()

In [ ]:
plt.plot(history_fine.history['accuracy'])
plt.plot(history_fine.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['training set','validation set'], loc='upper left')
plt.show()

In [ ]:
y_prob = model_cuisine.predict(X_test_bert_cuisine)
y_pred = np.argmax(y_prob,axis=1)
print(classification_report(y_test_int_cuisine,y_pred))

Indian Food Data

In [ ]:
model_RNN = RNNHyperModel(vocab_size_indian, 768, 2)

In [ ]:
tuner = kt.Hyperband(
    model_RNN,
    objective='val_accuracy',
    max_epochs=30,
    hyperband_iterations=2, 
    overwrite=True)

In [ ]:
tuner.search(X_train_bert_indian,y_train_indian,validation_data=(X_valid_bert_indian,y_valid_indian), epochs=10)

In [ ]:
bestHP = tuner.get_best_hyperparameters()[0]
print("[INFO] optimal number of filters in rnn1 layer: {}".format(bestHP.get("rnn1")))
print("[INFO] optimal number of units in rnn2 layer: {}".format(bestHP.get("rnn2")))
print("[INFO] optimal number of units in dense layer: {}".format(bestHP.get("dense_units")))
print("[INFO] optimal learning rate: {}".format(bestHP.get("learning_rt")))

In [ ]:
model_indian = tuner.get_best_models()[0]

In [ ]:
history_fine = model_indian.fit(X_train_bert_indian,y_train_indian,validation_data=(X_valid_bert_indian,y_valid_indian), epochs=10)

In [ ]:
plt.plot(history_fine.history['loss'])
plt.plot(history_fine.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['training set','validation set'], loc='upper left')
plt.show()

In [ ]:
plt.plot(history_fine.history['accuracy'])
plt.plot(history_fine.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['training set','validation set'], loc='upper left')
plt.show()

In [ ]:
y_prob = model_indian.predict(X_test_bert_indian)
y_pred = np.argmax(y_prob,axis=1)
print(classification_report(y_test_int_indian,y_pred))